# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`  
mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

***MongoDB has never worked correctly for me, despite multiple re-installations. I have to use MongoDBCompass to import files to databases.*** 

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
print(mongo.list_database_names())

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
print(db.list_collection_names())

In [ ]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_establishment = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new_establishment)

In [ ]:
# Check that the new restaurant was inserted

# Filter results by business name
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}
results = establishments.find(query, fields).limit(1)
pprint(results[0])

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = "1"

#Update only one record, the one for "Penang Flavours"
db.establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}})

In [ ]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
fields = {'BusinessName': 1, 'BusinessTypeID': 1, 'BusinessType': 1}
results = establishments.find(query, fields).limit(1)
print(results[0])

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Count documents before deletion
# Syntax provided by https://bootcampspot.instructure.com/courses/4981/external_tools/313

result = db.establishments.count_documents({})
print(result)

In [ ]:
# Count documents by 'LocalAuthorityName' before deletion

# Write aggregation query that counts number of documents, grouped by "LocalAuthorityName"
query = [{'$group': {'_id': '$LocalAuthorityName', 'count': { '$sum': 1 }}}]

# Run query with aggregate method and save results to variable
results = list(establishments.aggregate(query))

#Print results
pprint(results)

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
# Filter results by LocalAuthorityName = "Dover"
query = {'LocalAuthorityName': 'Dover'}
results = establishments.count_documents(query)
print(results)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}
establishments.delete_many(query)


In [ ]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1, 'BusinessName': 1,'LocalAuthorityName':1}
results = establishments.find(query, fields)
for result in results:
    pprint(result)

In [ ]:
#Count documents remaining in database
total_documents = establishments.count_documents({})
print(total_documents)

In [ ]:
# Check that other documents remain with 'find_one'
# use $ne operator, which stands for "not equal"
#query = {'LocalAuthorityName_1': {'$ne': 'Dover'}}
#results = establishments.find_one(query, fields)

result = db.establishments.find_one()
pprint(result)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
# PyMongo syntax provided by https://bootcampspot.instructure.com/courses/4981/external_tools/313

from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')

# Access the database
db = client['uk_food']

# Access the collection
collection = db['establishments']

#Access the geocode field
#geocode = data.get('geocode', {})

# Run the update query to convert latitude and longitude fields to double
result = collection.update_many(
    { "geocode.latitude": { "$exists": True }, "geocode.longitude": { "$exists": True } },
    [
        { 
            "$set": { 
                "geocode.latitude": { "$toDouble": "$geocode.latitude" },
                "geocode.longitude": { "$toDouble": "$geocode.longitude" }
            }
        }
    ]
)

print(f"Modified {result.modified_count} documents")


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# PyMongo syntax provided by https://bootcampspot.instructure.com/courses/4981/external_tools/313

# Access the database
db = client['uk_food']

# Access the collection
collection = db['establishments']

# Retrieve all documents from the collection
documents = collection.find({})

# Run the update query to convert the 'RatingValue' field to an integer
for doc in documents:
    # Get the current 'RatingValue' value
    current_rating_value = doc.get('RatingValue')
    
    # Check if 'RatingValue' exists and is a string
    if current_rating_value and isinstance(current_rating_value, str):
        try:
            # Try to convert the string value to an integer
            new_rating_value = int(current_rating_value)
        except ValueError:
            # Where the conversion fails, set new_rating_value to None
            #print(f"Could not convert '{current_rating_value}' to an integer for document with _id: {doc['_id']}")
            new_rating_value = None
        
        # Update the document with the new integer 'RatingValue'
        collection.update_many(
            {"_id": doc["_id"]},
            {"$set": {"RatingValue": new_rating_value}}
        )
        
print(f"Modified {result.modified_count} documents")

In [ ]:
# Set non 1-5 Rating Values to Null
#Previous query converted non 1-5 Rating Values to None, so 0 documents will be modified

# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')

# Access the database
db = client['uk_food']

# Access the collection
collection = db['establishments']

#Define non_ratings values
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]

#Run the update query convert non_ratings to Null
result = collection.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])                             
                                
print(f"Modified {result.modified_count} documents")

In [ ]:
# Check that the coordinates and rating value are now numbers
# PyMongo syntax provided by https://bootcampspot.instructure.com/courses/4981/external_tools/313

# Get a sample document from the collection
sample_document = collection.find_one()

# Print the 'geocode' field if it exists
if 'geocode' in sample_document:
    print("Geocode Field:")
    pprint(sample_document['geocode'])
else:
    print("No 'geocode' field found in the sample document.")

# Print the 'RatingValue' field if it exists
if 'RatingValue' in sample_document:
    print("\nRatingValue Field:")
    pprint(sample_document['RatingValue'])
else:
    print("No 'RatingValue' field found in the sample document.")